In [1]:
import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd

# 1. 리뷰 스크래핑

In [2]:
# 잡플래닛 로그인 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

In [3]:
# 1.정부/공공기관/공기업(1001), 2.은행/금융업(900), 3.네트워크/통신/모바일(704), 4.세무/회계(1004)
codes = ['1001', '900', '704', '1004']
nums = ['1', '2', '3', '4']

In [4]:
# PyKoSpacing(띄어쓰기 교정) 패키지 적용
from pykospacing import Spacing
spacing = Spacing()

In [6]:
f = open('review_train.txt', 'w', encoding = 'utf-8') # 리뷰를 txt파일로 저장

for code, count in zip(codes, nums):
    reviews = {}
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math
    pages = math.ceil(num / 10) # 한 페이지당 리뷰 10개로 구성됨
    
    for i in range(1, pages + 1): # 전체 페이지의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_ = 'us_label') # 한 줄 리뷰 추출
        labels = [label.get_text().replace('\r', ' ') for label in label[:]] # 텍스트만 추출하여 labels 리스트에 저장  
        star = soup.find_all('div', class_="us_star_m") # 별점이 포함된 div 추출
        stars = re.findall('[0-9]+[.]+[0-9]', str(star)) # 별점을 나타내는 width 속성의 숫자 부분을 stars 리스트에 저장

        # txt파일에 리뷰 쓰기
        for k in range(len(labels)):
            start = labels[k].find('"')
            f.write(spacing(labels[k][start+1:-1]) + '  ')
            # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
            reviews[spacing(labels[k][start+1:-1])] = stars[k].replace('.0', '')
 
    # 동적변수를 할당하여 산업군별 리뷰를 각각 리스트로 저장(총 4개의 딕셔너리 생성)
    globals()['reviews' + str(count)] = reviews
    f.write('\n')
f.close()

In [7]:
# 산업군별 리뷰 4개의 딕셔너리를 하나로 합치기
reviews_all = {**reviews1, **reviews2, **reviews3, **reviews4}

In [9]:
# 딕셔너리를 pickle 파일로 저장
import pickle
with open('reviews_all.pkl', 'wb') as f:
    pickle.dump(reviews_all, f)
with open('reviews1.pkl', 'wb') as f:
    pickle.dump(reviews1, f)
with open('reviews2.pkl', 'wb') as f:
    pickle.dump(reviews2, f)
with open('reviews3.pkl', 'wb') as f:
    pickle.dump(reviews3, f)
with open('reviews4.pkl', 'wb') as f:
    pickle.dump(reviews4, f)

In [13]:
len(reviews_all)

2558

In [14]:
# 각 산업군별 리뷰 개수 확인
print(len(reviews1), len(reviews2), len(reviews3), len(reviews4))

775 1065 343 375


###  review_train.txt는 하나의 산업군 리뷰 전체를 한 줄로 스크래핑하여 총 4문장으로 이루어져 있음
 - 각 리뷰는 '  '로 구분하여 저장하였음.
 - -> 이는 soynlp의 클래스 사용 시 적합하도록 저장한 것임.
 - (한 줄을 하나의 문서로 인식, 한 문서 내에서 두 칸 띄어쓰기로 문장 구분)

# 2. soynlp를 이용한 명사 추출기 적용
#### 목적: 말뭉치를 기반으로 학습된 기존의 형태소 분석기는 pos 태깅 시 복합명사를 각각의 명사로 태깅하거나 
#### 새로 등장하는 단어를 명사로 인식하지 못하는 문제가 있음. 이에 따라 문서집합의 학습을 통해 명사점수를 부여하고 명사가 아닌지 판단
 - L - [R]의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단
 - 해당 패키지는 세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set에서 명사 가능 점수가 학습되어 있음.
 - 명사 가능 점수의 범위: [-1, 1]
 - 통계 기반의 unsupervised 학습 방법 적용
 
#### 두 가지의 명사 추출기를 적용하여 봄

In [3]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [17]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'data/review_train.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

2564

## 1) LRNounExtractor
#### soynlp의 기본적인 명사 추출기 적용

In [12]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


#### 전체 리뷰 문장에서 단어를 추출하여 명사를 판단하는 학습 실행

In [18]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (654, 331) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 159 nouns are extracted
Wall time: 633 ms


In [19]:
# 빈도수 기준 상위 100개 단어
sort_nouns1 = sorted(nouns.items(), key=lambda x:-x[1].frequency)[:100]
print(sort_nouns1)

[('회사', NounScore_v1(frequency=728, score=0.6604329772727273, known_r_ratio=0.7553648068669528)), ('업무', NounScore_v1(frequency=368, score=0.8664290380434783, known_r_ratio=0.9246231155778895)), ('사람', NounScore_v1(frequency=203, score=0.5184580439560439, known_r_ratio=0.621160409556314)), ('기업', NounScore_v1(frequency=193, score=0.8675755227272727, known_r_ratio=0.6567164179104478)), ('다양', NounScore_v1(frequency=145, score=0.9994672624113472, known_r_ratio=0.9724137931034482)), ('사람들', NounScore_v1(frequency=140, score=0.9229012244897958, known_r_ratio=0.98989898989899)), ('복지', NounScore_v1(frequency=132, score=0.9073422142857143, known_r_ratio=0.875)), ('추천', NounScore_v1(frequency=131, score=0.9868988666666667, known_r_ratio=0.28846153846153844)), ('생각', NounScore_v1(frequency=129, score=0.9089652835820896, known_r_ratio=0.5775862068965517)), ('경험', NounScore_v1(frequency=129, score=0.646697743902439, known_r_ratio=0.7387387387387387)), ('직원', NounScore_v1(frequency=114, score=0.4

In [20]:
# 빈도수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns1):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.frequency), end = '')


    회사 (728.00)    업무 (368.00)    사람 (203.00)    기업 (193.00)    다양 (145.00)
   사람들 (140.00)    복지 (132.00)    추천 (131.00)    생각 (129.00)    경험 (129.00)
    직원 (114.00)    연봉 (112.00)   직원들 (101.00)    문화 (99.00)    직장 (96.00)
    근무 (96.00)    성장 (92.00)    급여 (82.00)   안정적 (82.00)    경력 (81.00)
    발전 (71.00)    체계 (65.00)    가능 (64.00)    환경 (62.00)   공무원 (61.00)
    최고 (58.00)    하나 (57.00)    좋아 (56.00)    조직 (56.00)    개인 (56.00)
   계약직 (56.00)    신입 (55.00)    사업 (54.00)    시간 (54.00)    자유 (53.00)
    본인 (53.00)    강도 (52.00)   보수적 (52.00)    대우 (51.00)   워라밸 (51.00)
    개발 (50.00)    장점 (48.00)    정도 (48.00)  공공기관 (48.00)    야근 (46.00)
   사무실 (45.00)    분들 (44.00)    운영 (42.00)    위치 (42.00)   커리어 (42.00)
    무난 (41.00)    이상 (41.00)   가능성 (41.00)    능력 (40.00)    수준 (40.00)
    업계 (40.00)   수평적 (40.00)    기회 (39.00)   워라벨 (38.00)    부서 (37.00)
    규모 (37.00)    가족 (37.00)    대표 (36.00)   정규직 (36.00)  스트레스 (36.00)
    노력 (35.00)    단점 (35.00)    느낌 (35.00)   공기업 (35.00)    입사 

In [21]:
# 빈도수 * 명사점수 기준 상위 100개 단어
sort_nouns2 = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]
print(sort_nouns2)

[('회사', NounScore_v1(frequency=728, score=0.6604329772727273, known_r_ratio=0.7553648068669528)), ('업무', NounScore_v1(frequency=368, score=0.8664290380434783, known_r_ratio=0.9246231155778895)), ('기업', NounScore_v1(frequency=193, score=0.8675755227272727, known_r_ratio=0.6567164179104478)), ('다양', NounScore_v1(frequency=145, score=0.9994672624113472, known_r_ratio=0.9724137931034482)), ('추천', NounScore_v1(frequency=131, score=0.9868988666666667, known_r_ratio=0.28846153846153844)), ('사람들', NounScore_v1(frequency=140, score=0.9229012244897958, known_r_ratio=0.98989898989899)), ('복지', NounScore_v1(frequency=132, score=0.9073422142857143, known_r_ratio=0.875)), ('생각', NounScore_v1(frequency=129, score=0.9089652835820896, known_r_ratio=0.5775862068965517)), ('사람', NounScore_v1(frequency=203, score=0.5184580439560439, known_r_ratio=0.621160409556314)), ('직원들', NounScore_v1(frequency=101, score=0.8547858363636363, known_r_ratio=1.0)), ('경험', NounScore_v1(frequency=129, score=0.64669774390243

In [22]:
# 빈도수 * 명사점수 기준 상위 100개 단어 확인
for i, (word, score) in enumerate(sort_nouns2):
    if i % 5 ==0:
        print()
    print('%6s (%.2f)' % (word, score.score), end = '')


    회사 (0.66)    업무 (0.87)    기업 (0.87)    다양 (1.00)    추천 (0.99)
   사람들 (0.92)    복지 (0.91)    생각 (0.91)    사람 (0.52)   직원들 (0.85)
    경험 (0.65)   안정적 (1.00)    문화 (0.82)    성장 (0.81)    급여 (0.89)
    연봉 (0.61)    직장 (0.67)    가능 (0.99)    근무 (0.65)    경력 (0.73)
    최고 (1.00)    직원 (0.49)    좋아 (0.99)    체계 (0.85)    자유 (0.99)
   보수적 (1.00)    조직 (0.93)   공무원 (0.84)    본인 (0.95)    신입 (0.91)
    개인 (0.88)    환경 (0.76)    분들 (0.97)  공공기관 (0.88)   커리어 (1.00)
   계약직 (0.73)    무난 (1.00)   사무실 (0.91)   수평적 (1.00)    업계 (1.00)
    위치 (0.91)    대우 (0.75)    운영 (0.90)    시간 (0.70)    야근 (0.81)
    부서 (1.00)    하나 (0.63)    규모 (0.94)    발전 (0.48)    기회 (0.88)
    느낌 (0.96)    부족 (0.99)    필요 (0.99)    입사 (0.99)    노력 (0.91)
   전반적 (1.00)    기관 (1.00)    성과 (1.00)    능력 (0.77)    관련 (1.00)
   공기업 (0.88)    이상 (0.74)    정도 (0.63)    나름 (1.00)    대표 (0.82)
    사업 (0.54)    전문 (1.00)    강도 (0.55)    은행 (0.93)    투자 (0.99)
    계속 (1.00)    장점 (0.57)    처음 (1.00)    관심 (1.00)    최악 (0.95)
    꼰대 (0

## 2) NewsNounExtractor
#### 뉴스 데이터에서 더 좋은 성능을 내기 위해 여러가지 후처리 과정이 추가된 명사 추출기 적용

In [23]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/hsmy1/Anaconda3/envs/py37/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [24]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (20785, 11074, 11995)
predicting noun score was done                                        
before postprocessing 3495
_noun_scores_ 1043
checking hardrules ... done / 1043떡볶+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 653
extracted 0 compounds from eojeolss ... 1000 / 1094

In [25]:
# 각 단어별 점수 및 빈도수 확인
words = ['공공기관', '워라밸', '분위기', '야근', '꼰대']

for word in words:
    if not word in nouns:
        continue
    score = nouns[word]
    print('word: %s, score: %.2f, frequency: %.2f' % (word, score.score, score.frequency))

word: 공공기관, score: 0.91, frequency: 63.00
word: 분위기, score: 0.49, frequency: 260.00
word: 야근, score: 0.81, frequency: 51.00
word: 꼰대, score: 0.76, frequency: 32.00


In [10]:
# 리뷰 개수가 적은 산업군을 감안하여 최소 빈도수를 15로 조정
def nounextractor(sent):
    noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
    )
    
    nouns = noun_extractor.train_extract(
        sent,
        min_noun_score=0.3,
        min_noun_frequency=15
    )

    # 빈도수 * 명사점수 기준 상위 100개 단어
    sort_nouns = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]

    for i, (word, score) in enumerate(sort_nouns):
        if i % 5 ==0:
            print()
        print('%6s (%.2f, %.2f)' % (word, score.frequency, score.score), end = '')

In [27]:
nounextractor(sentences)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (854, 432) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 226 nouns are extracted

    회사 (732.00, 0.66)    업무 (368.00, 0.87)    기업 (194.00, 0.87)    다양 (145.00, 1.00)    추천 (131.00, 0.99)
   사람들 (140.00, 0.92)    복지 (133.00, 0.91)    생각 (130.00, 0.91)    사람 (209.00, 0.53)   직원들 (101.00, 0.85)
    경험 (130.00, 0.65)   안정적 (82.00, 1.00)    문화 (99.00, 0.82)    성장 (93.00, 0.82)    급여 (83.00, 0.89)
    연봉 (112.00, 0.61)    근무 (98.00, 0.66)    가능 (65.00, 0.99)    직장 (96.00, 0.67)    경력 (83.00, 0.73)
    최고 (58.00, 1.00)    직원 (115.00, 0.50)    좋아 (56.00, 0.99)   보수적 (54.00, 1.00)   공무원 (63.00, 0.85)
    자유 (53.00, 0.99)    신입 (57.00, 0.92)    조직 (56.00, 0.93)    본인 (53.00, 0.95)    환경 (62.00, 0.76)
  공공기관 (52.00, 0.89)    체계 (54.00, 0.85)   계약직 (58.00, 0.74)    분들 (4

### 전체 산업군과 별도로 각각의 산업군별으로도 분석해보기 위해 텍스트 파일을 4개로 분할


In [1]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f :
    data = f.read().split('\n')
    for i in range(4):
        a = 1 + i
        with open(f'review_{a}.txt', 'w+', encoding = 'utf-8') as txt:
            txt.write(data[i]) 

In [18]:
corpus_fname1 = 'data/review_1.txt'
sentences1 = DoublespaceLineCorpus(corpus_fname1, iter_sent=True)
len(sentences1)

775

In [6]:
corpus_fname2 = 'data/review_2.txt'
sentences2 = DoublespaceLineCorpus(corpus_fname2, iter_sent=True)
len(sentences2)

1068

In [7]:
corpus_fname3 = 'data/review_3.txt'
sentences3 = DoublespaceLineCorpus(corpus_fname3, iter_sent=True)
len(sentences3)

346

In [8]:
corpus_fname4 = 'data/review_4.txt'
sentences4 = DoublespaceLineCorpus(corpus_fname4, iter_sent=True)
len(sentences4)

375

### 각 산업군별 빈도수 * 명사점수 기준 상위 100개 단어 확인

In [13]:
nounextractor(sentences1)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (274, 138) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 57 nouns are extracted

    업무 (130.00, 0.85)    다양 (40.00, 1.00)   공무원 (50.00, 0.80)    회사 (76.00, 0.49)    복지 (38.00, 0.98)
   계약직 (44.00, 0.82)    경험 (53.00, 0.68)    근무 (47.00, 0.74)   사람들 (39.00, 0.87)    생각 (38.00, 0.88)
  공공기관 (38.00, 0.84)    기업 (31.00, 1.00)    추천 (30.00, 0.99)   안정적 (27.00, 1.00)    기관 (26.00, 1.00)
    가능 (25.00, 0.99)    조직 (27.00, 0.92)    급여 (26.00, 0.92)    직장 (39.00, 0.57)   공기업 (23.00, 0.95)
    사람 (55.00, 0.38)    환경 (25.00, 0.80)    강도 (22.00, 0.90)   직원들 (21.00, 0.86)    경력 (21.00, 0.85)
    서울 (17.00, 1.00)    관련 (16.00, 1.00)    문화 (24.00, 0.66)    만족 (16.00, 0.99)    좋아 (16.00, 0.98)
    문제 (15.00, 1.00)    체계 (19.00, 0.76)    개인 (14.00, 1.00)   정규직 (19.00, 0.73)  

In [14]:
nounextractor(sentences2)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (397, 200) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 86 nouns are extracted

    회사 (400.00, 0.65)    업무 (140.00, 0.90)   분위기 (101.00, 1.00)    기업 (101.00, 0.88)   사람들 (66.00, 0.94)
    문화 (58.00, 0.87)    다양 (49.00, 1.00)    복지 (52.00, 0.94)    추천 (49.00, 0.98)    생각 (52.00, 0.92)
    성장 (57.00, 0.82)    연봉 (67.00, 0.66)    사람 (71.00, 0.61)   안정적 (42.00, 1.00)   직원들 (48.00, 0.79)
    최고 (36.00, 1.00)   보수적 (35.00, 1.00)    급여 (37.00, 0.87)    은행 (30.00, 0.93)    투자 (26.00, 0.99)
    본인 (26.00, 0.98)    업계 (25.00, 1.00)   커리어 (24.00, 1.00)    기회 (24.00, 1.00)    개인 (23.00, 1.00)
    조직 (25.00, 0.92)    규모 (25.00, 0.91)    경험 (37.00, 0.62)    직원 (41.00, 0.54)    체계 (23.00, 0.96)
    무난 (22.00, 1.00)    정도 (22.00, 0.99)    가능 (22.00, 0.99)    성과 (21.00, 1.00

In [15]:
nounextractor(sentences3)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (114, 63) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 16 nouns are extracted

    회사 (137.00, 0.77)    기업 (46.00, 0.71)    업무 (42.00, 0.74)    다양 (21.00, 1.00)    복지 (20.00, 0.96)
    직원 (20.00, 0.73)    개발 (19.00, 0.75)    경험 (21.00, 0.67)   직원들 (14.00, 0.92)    성장 (17.00, 0.67)
    생각 (12.00, 0.95)    사람 (23.00, 0.48)    대우 (10.00, 1.00)    연봉 (18.00, 0.53)    사업 (12.00, 0.59)
    근무 (8.00, 0.63)

In [16]:
nounextractor(sentences4)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (127, 71) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 19 nouns are extracted

    회사 (98.00, 0.54)    업무 (52.00, 0.91)    다양 (33.00, 1.00)    사람 (45.00, 0.70)   사무실 (27.00, 1.00)
    신입 (25.00, 0.99)   사람들 (22.00, 0.97)    복지 (21.00, 1.00)    추천 (18.00, 0.99)   직원들 (18.00, 0.95)
   회계사 (14.00, 0.98)    직원 (32.00, 0.42)    야근 (15.00, 0.84)    경력 (16.00, 0.75)   거래처 (16.00, 0.74)
    경험 (18.00, 0.58)    입사 (10.00, 0.99)    생각 (7.00, 0.80)    개인 (12.00, 0.34)